In [13]:
import numpy as np
import torch
import torch.nn.functional as F
import mmcv
from mmdet.models import SingleStageDetector
from mmdet.apis import set_random_seed
from mmcv import Config
from mmdet.apis import init_detector, inference_detector

In [14]:

cfg = Config.fromfile('/home/anna/Загрузки/mmdetection/configs/ssd/ssd300_coco.py')
cfg.model['backbone'] = dict(
        type='EfficientNet', # Using MobileNetV3 from mmcls
        arch='b3',# Modify out_indices
        
        drop_path_rate=0.0, out_indices=(3,), #frozen_stages=3, 
        conv_cfg={'type': 'Conv2dAdaptivePadding'}, 
        norm_cfg={'eps': 0.001, 'type': 'BN'}, 
        act_cfg={'type': 'Swish'}, norm_eval=False, with_cp=False, 
        init_cfg=[{'type': 'Kaiming', 'layer': 'Conv2d'}, {'type': 'Constant', 'layer': ['_BatchNorm', 'GroupNorm'], 'val': 1},])

cfg.data_root = '/home/anna/Загрузки/mmdetection/data/coco/'

cfg.data.train.dataset.data_root = '/home/anna/Загрузки/mmdetection/'
cfg.data.train.dataset.ann_file = '/home/anna/Загрузки/mmdetection/data/coco/annotations/instances_train2017.json'


cfg.load_from = '/home/anna/MFTI/data/COCO/person_coco_ssd_eefic_.pth'


cfg.data.val.ann_file = '/home/anna/Загрузки/mmdetection/data/coco/annotations/instances_val2017.json'
cfg.model.bbox_head.num_classes = 1
# Set up working dir to save files and logs.
cfg.work_dir = '/home/anna/MFTI/data/COCO/tutorial_exps'


cfg.log_config.interval = 100

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 100
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 100


# Set seed thus the results are more reproducible
cfg.seed = 42
set_random_seed(42, deterministic=False)
cfg.gpu_ids = range(1)
cfg['device'] = 'cpu'
# We can initialize the logger for training and have a look
# at the final config used for training

print(f'Config:\n{cfg.pretty_text}')

Config:
input_size = 300
model = dict(
    type='SingleStageDetector',
    backbone=dict(
        type='EfficientNet',
        arch='b3',
        drop_path_rate=0.0,
        out_indices=(3, ),
        conv_cfg=dict(type='Conv2dAdaptivePadding'),
        norm_cfg=dict(eps=0.001, type='BN'),
        act_cfg=dict(type='Swish'),
        norm_eval=False,
        with_cp=False,
        init_cfg=[
            dict(type='Kaiming', layer='Conv2d'),
            dict(type='Constant', layer=['_BatchNorm', 'GroupNorm'], val=1)
        ]),
    neck=dict(
        type='SSDNeck',
        in_channels=(512, 1024),
        out_channels=(512, 1024, 512, 256, 256, 256),
        level_strides=(2, 2, 1, 1),
        level_paddings=(1, 1, 0, 0),
        l2_norm_scale=20),
    bbox_head=dict(
        type='SSDHead',
        in_channels=(512, 1024, 512, 256, 256, 256),
        num_classes=1,
        anchor_generator=dict(
            type='SSDAnchorGenerator',
            scale_major=False,
            input_siz

In [3]:
checkpoint = '/home/anna/Загрузки/checkpoints/ssd300_coco_20210803_015428-d231a06e.pth'

In [4]:
# Set optimizer
cfg.optimizer = dict(type='SGD', lr=0.00045, momentum=0.9, weight_decay=0.0001)
cfg.optimizer_config = dict(grad_clip=dict(max_norm=35, norm_type=2))
cfg.runner = dict(type='EpochBasedRunner', max_epochs=1)

![](/home/anna/Загрузки/mmdetection/data/coco/train2017/000000001407.jpg)

In [4]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
cfg.data.workers_per_gpu=0

# Set optimizer
cfg.optimizer = dict(type='SGD', lr=0.002, momentum=0.9, weight_decay=0.0001)
cfg.optimizer_config = dict(grad_clip=dict(max_norm=35, norm_type=2))
# Set customized learning policy
cfg.lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=0.0000000001,
    step=[8])
cfg.runner = dict(type='EpochBasedRunner', max_epochs=10)

model = build_detector( cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
datasets = [build_dataset(cfg.data.train)]

train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=18.96s)
creating index...
index created!


2023-01-31 11:13:56,560 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.


loading annotations into memory...


2023-01-31 11:13:59,714 - mmdet - INFO - load checkpoint from local path: /home/anna/MFTI/data/COCO/person_coco_ssd_eefic_.pth


Done (t=2.89s)
creating index...
index created!


...

2023-02-01 04:11:38,800 - mmdet - INFO - Epoch [8][100/1010]	lr: 2.000e-03, eta: 6:51:47, time: 8.345, data_time: 0.310, loss_cls: 2.3078, loss_bbox: 3.2419, loss: 5.5497, grad_norm: 2.9883
2023-02-01 04:25:41,360 - mmdet - INFO - Epoch [8][200/1010]	lr: 2.000e-03, eta: 6:37:43, time: 8.426, data_time: 0.299, loss_cls: 2.3169, loss_bbox: 3.2256, loss: 5.5425, grad_norm: 2.9618
2023-02-01 04:39:36,264 - mmdet - INFO - Epoch [8][300/1010]	lr: 2.000e-03, eta: 6:23:37, time: 8.349, data_time: 0.288, loss_cls: 2.3110, loss_bbox: 3.2047, loss: 5.5157, grad_norm: 3.1136
2023-02-01 04:53:20,190 - mmdet - INFO - Epoch [8][400/1010]	lr: 2.000e-03, eta: 6:09:27, time: 8.239, data_time: 0.294, loss_cls: 2.3119, loss_bbox: 3.3812, loss: 5.6932, grad_norm: 3.0860
2023-02-01 05:07:12,692 - mmdet - INFO - Epoch [8][500/1010]	lr: 2.000e-03, eta: 5:55:21, time: 8.325, data_time: 0.291, loss_cls: 2.3022, loss_bbox: 3.3299, loss: 5.6321, grad_norm: 2.8008
2023-02-01 05:21:02,672 - mmdet - INFO - Epoch [8]

train


2023-02-01 06:31:52,613 - mmdet - INFO - Epoch [9][100/1010]	lr: 2.000e-04, eta: 4:29:06, time: 8.279, data_time: 0.311, loss_cls: 2.2930, loss_bbox: 3.2256, loss: 5.5186, grad_norm: 2.7256
2023-02-01 06:46:19,099 - mmdet - INFO - Epoch [9][200/1010]	lr: 2.000e-04, eta: 4:15:11, time: 8.665, data_time: 0.311, loss_cls: 2.2828, loss_bbox: 3.3309, loss: 5.6136, grad_norm: 2.6482
2023-02-01 07:00:09,979 - mmdet - INFO - Epoch [9][300/1010]	lr: 2.000e-04, eta: 4:01:07, time: 8.309, data_time: 0.295, loss_cls: 2.2839, loss_bbox: 3.2839, loss: 5.5678, grad_norm: 2.8238
2023-02-01 07:14:00,523 - mmdet - INFO - Epoch [9][400/1010]	lr: 2.000e-04, eta: 3:47:04, time: 8.305, data_time: 0.294, loss_cls: 2.2839, loss_bbox: 3.2825, loss: 5.5664, grad_norm: 2.5596
2023-02-01 07:28:05,446 - mmdet - INFO - Epoch [9][500/1010]	lr: 2.000e-04, eta: 3:33:04, time: 8.449, data_time: 0.302, loss_cls: 2.2848, loss_bbox: 3.2490, loss: 5.5337, grad_norm: 2.8957
2023-02-01 07:42:09,068 - mmdet - INFO - Epoch [9]

train


2023-02-01 08:56:14,540 - mmdet - INFO - Epoch [10][100/1010]	lr: 2.000e-04, eta: 2:07:40, time: 8.747, data_time: 0.327, loss_cls: 2.2846, loss_bbox: 3.2114, loss: 5.4960, grad_norm: 2.7100
2023-02-01 09:10:41,787 - mmdet - INFO - Epoch [10][200/1010]	lr: 2.000e-04, eta: 1:53:41, time: 8.672, data_time: 0.306, loss_cls: 2.2814, loss_bbox: 3.1252, loss: 5.4066, grad_norm: 2.8084
2023-02-01 09:24:16,905 - mmdet - INFO - Epoch [10][300/1010]	lr: 2.000e-04, eta: 1:39:37, time: 8.151, data_time: 0.282, loss_cls: 2.2865, loss_bbox: 3.2399, loss: 5.5264, grad_norm: 2.7433
2023-02-01 09:38:03,370 - mmdet - INFO - Epoch [10][400/1010]	lr: 2.000e-04, eta: 1:25:34, time: 8.265, data_time: 0.299, loss_cls: 2.2780, loss_bbox: 3.2149, loss: 5.4929, grad_norm: 2.5001
2023-02-01 09:51:32,623 - mmdet - INFO - Epoch [10][500/1010]	lr: 2.000e-04, eta: 1:11:30, time: 8.093, data_time: 0.288, loss_cls: 2.2793, loss_bbox: 3.2439, loss: 5.5231, grad_norm: 2.5114
2023-02-01 10:04:54,507 - mmdet - INFO - Epoc

In [5]:
import pickle
torch.save(model.state_dict(), '/home/anna/MFTI/data/person_coco_ssd_2.pth')
pickle.dump(model, open ('/home/anna/MFTI/data/ssd_model.pkl', 'wb'))